# Create TF.Dataset

In [1]:
!pip install -qU pip
!pip install -qU tensorflow-gpu==2.0.0-alpha0

In [2]:
import tensorflow as tf

In [3]:
!head -n5 ../input/train.csv

In [4]:
column_names = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
select_columns = ['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
label_name = 'Survived'

In [5]:
train_dataset = tf.data.experimental.make_csv_dataset(
    file_pattern = '../input/train.csv',
    batch_size = 32,
    column_names = column_names,
    select_columns = select_columns,
    label_name = label_name,
    num_epochs = 1)

In [6]:
features, labels = next(iter(train_dataset))
print(features)

OrderedDict([('Pclass', <tf.Tensor: id=78, shape=(32,), dtype=int32, numpy=
array([3, 3, 2, 3, 3, 3, 3, 2, 3, 1, 3, 2, 3, 2, 3, 3, 3, 3, 2, 1, 3, 3,
       1, 1, 3, 3, 3, 3, 1, 1, 3, 3], dtype=int32)>), ('Sex', <tf.Tensor: id=79, shape=(32,), dtype=string, numpy=
array([b'male', b'female', b'male', b'male', b'female', b'male', b'male',
       b'male', b'male', b'female', b'male', b'male', b'female', b'male',
       b'female', b'male', b'male', b'male', b'male', b'male', b'male',
       b'male', b'male', b'female', b'female', b'male', b'male',
       b'female', b'female', b'female', b'female', b'male'], dtype=object)>), ('Age', <tf.Tensor: id=74, shape=(32,), dtype=float32, numpy=
array([ 0.  , 45.  , 32.  ,  0.  ,  0.  , 20.  , 30.  , 66.  , 45.  ,
       15.  , 30.  , 31.  ,  0.75, 16.  ,  0.  ,  0.  , 36.  ,  9.  ,
       46.  , 26.  , 22.  , 20.5 ,  0.  ,  0.  ,  0.  ,  4.  , 32.  ,
        0.  , 21.  , 30.  , 14.  ,  0.  ], dtype=float32)>), ('SibSp', <tf.Tensor: id=80, shape=(32,)

# Feature Engineering

In [7]:
from tensorflow import feature_column
from tensorflow.keras import layers

In [8]:
feature_columns = []

raw = feature_column.categorical_column_with_identity('Pclass', num_buckets=4)
feature_columns.append(feature_column.indicator_column(raw))
raw = feature_column.categorical_column_with_vocabulary_list('Sex', ['male', 'female'])
feature_columns.append(feature_column.indicator_column(raw))
feature_columns.append(feature_column.numeric_column('Age'))
raw = feature_column.categorical_column_with_identity('SibSp', num_buckets=9)
feature_columns.append(feature_column.indicator_column(raw))
raw = feature_column.categorical_column_with_identity('Parch', num_buckets=12)
feature_columns.append(feature_column.indicator_column(raw))
feature_columns.append(feature_column.numeric_column('Fare'))
raw = feature_column.categorical_column_with_vocabulary_list('Embarked', ['S', 'C', 'Q'])
feature_columns.append(feature_column.indicator_column(raw))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

#Model

In [9]:
model = tf.keras.models.Sequential([
    feature_layer,
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5), 
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [10]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [11]:
model.fit(train_dataset, epochs=100)

W0504 04:08:40.145957 139897865573760 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/feature_column/feature_column_v2.py:2758: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0504 04:08:40.163760 139897865573760 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/lookup_ops.py:1347: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0504 04:08:40.171941 139897865573760 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please

Epoch 1/100
28/28 [==============================] - 3s 104ms/step - loss: 1.5162 - accuracy: 0.5844
Epoch 2/100
28/28 [==============================] - 1s 39ms/step - loss: 1.3229 - accuracy: 0.5888
Epoch 3/100
28/28 [==============================] - 1s 45ms/step - loss: 1.1922 - accuracy: 0.6053
Epoch 4/100
28/28 [==============================] - 1s 48ms/step - loss: 0.9465 - accuracy: 0.6482
Epoch 5/100
28/28 [==============================] - 1s 46ms/step - loss: 0.8316 - accuracy: 0.6578
Epoch 6/100
28/28 [==============================] - 1s 42ms/step - loss: 0.8251 - accuracy: 0.6946
Epoch 7/100
28/28 [==============================] - 1s 39ms/step - loss: 0.7864 - accuracy: 0.6937
Epoch 8/100
28/28 [==============================] - 1s 40ms/step - loss: 0.7407 - accuracy: 0.7178
Epoch 9/100
28/28 [==============================] - 1s 40ms/step - loss: 0.6804 - accuracy: 0.7193
Epoch 10/100
28/28 [==============================] - 1s 39ms/step - loss: 0.6426 - accuracy: 0.693

In [12]:
test_column_names = ['PassengerId','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
test_select_columns = ['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

test_dataset = tf.data.experimental.make_csv_dataset(
    shuffle = False,
    file_pattern = '../input/test.csv',
    batch_size = 64,
    column_names = test_column_names,
    select_columns = test_select_columns,
    num_epochs = 1)

In [13]:
features = next(iter(test_dataset))
print(features)

OrderedDict([('PassengerId', <tf.Tensor: id=1541350, shape=(64,), dtype=int32, numpy=
array([892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904,
       905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917,
       918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930,
       931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943,
       944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955],
      dtype=int32)>), ('Pclass', <tf.Tensor: id=1541351, shape=(64,), dtype=int32, numpy=
array([3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 1, 1, 2, 1, 2, 2, 3, 3, 3, 1, 3,
       1, 1, 1, 3, 1, 3, 1, 3, 2, 2, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 2,
       1, 3, 1, 3, 1, 3, 1, 2, 2, 1, 2, 3, 3, 3, 3, 1, 3, 2, 3, 3],
      dtype=int32)>), ('Sex', <tf.Tensor: id=1541352, shape=(64,), dtype=string, numpy=
array([b'male', b'female', b'male', b'male', b'female', b'male',
       b'female', b'male', b'female', b'male', b'male', b'male',
       b'female', b'male', b'female'

In [14]:
predictions = model.predict(test_dataset) > 0.5

In [15]:
predictions[:, 0]

array([False, False, False, False, False, False, False, False,  True,
       False, False, False,  True, False,  True,  True, False, False,
       False, False, False, False,  True, False,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False,  True,  True,
       False, False, False,  True,  True, False, False,  True,  True,
        True, False, False, False, False,  True, False, False, False,
        True,  True,  True,  True, False, False,  True,  True, False,
        True, False,  True, False, False,  True, False,  True,  True,
       False, False, False, False, False, False,  True,  True,  True,
       False, False,  True, False,  True, False,  True, False,  True,
       False,  True, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True, False, False,
        True, False,  True,  True, False,  True, False, False,  True,
       False,  True,

In [16]:
import pandas as pd

In [17]:
test_df = pd.read_csv('../input/test.csv')

In [18]:
final_df = pd.DataFrame()

In [19]:
final_df['PassengerId'] = test_df['PassengerId']

In [20]:
final_df['Survived'] = predictions[:, 0]*1

In [21]:
final_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [22]:
final_df.to_csv('submission.csv', index=False)

In [23]:
!head -n5 submission.csv

PassengerId,Survived
892,0
893,0
894,0
895,0
